In [1]:
import wrds
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
import sys
from pathlib import Path

# add the src directory to the path in order to import config
current_directory = Path.cwd()
src_path = current_directory.parent / "src"
sys.path.insert(0, str(src_path))

import load_option_data_01 as l1
import filter_option_data_01 as f1
import config
WRDS_USERNAME = Path(config.WRDS_USERNAME)
DATA_DIR = Path(config.DATA_DIR)

In [2]:
import bsm_pricer

In [3]:
def pull_option_price(wrds_username, year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	sql_query = f"""
					SELECT 
						*
					FROM
						optionm_all.opprcd{year} AS a
					WHERE
						a.secid=108105;
				"""
		
	optm_df = db.raw_sql(sql_query, date_cols = ["date"])

	db.close()

	return optm_df

def pull_all_option_price(wrds_username, last_year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	optm_df = []

	for year in range(1996,last_year+1):
		sql_query = f"""
						SELECT 
							a.date, a.exdate, a.cp_flag, a.strike_price
						FROM
							optionm_all.opprcd{year} AS a
						WHERE
							a.secid=108105;
					"""
		
		optm = db.raw_sql(sql_query, date_cols = ["date"])

		optm_df.append(optm)
		
	db.close()

	optm_df = pd.concat(optm_df)
	
	return optm_df


def pull_all_req_data(wrds_username, start_year=1996, end_year=2012): 
	db = wrds.Connection(wrds_username=wrds_username)

	optm_df = []

	for year in range(start_year, end_year+1):
		sql_query = f"""
						SELECT 
							a.date, a.exdate, a.cp_flag, a.strike_price, a.best_bid, a.best_offer,a.volume,
							b.open, b.close, a.impl_volatility, c.tb_m3
						FROM
							optionm_all.opprcd{year} AS a
						JOIN 
							optionm_all.secprd{year} AS b ON a.date = b.date AND a.secid = b.secid
						JOIN 
							frb_all.rates_daily AS c ON c.date = a.date 
						WHERE
							a.secid=108105 AND a.date <= '2012-01-31';
					"""
		
		optm = db.raw_sql(sql_query, date_cols = ["date"])

		optm_df.append(optm)
		
	db.close()

	optm_df = pd.concat(optm_df)
	
	return optm_df

def pull_Opt_Sec_info_WRDS(wrds_username = WRDS_USERNAME, start = '1996-01-04', end = '2012-01-31'): 
	#https://wrds-www.wharton.upenn.edu/pages/get-data/option-suite-wrds/us-option-level-output/
	sql_query = f"""
		SELECT  
			a.*, c.tb_m3
		FROM
			beta.wrdsapps_optionsig  AS a
		JOIN 
			frb_all.rates_daily AS c ON c.date = a.date 
		WHERE
			(a.secid = 108105) 
		AND 
			(a.date <= \'{end}\') 
		AND 
			(a.date >= \'{start}\')
		LIMIT 1000
	""" 
	#LIMIT 1000
	db = wrds.Connection(wrds_username=wrds_username)
	optm = db.raw_sql(sql_query, date_cols = ["date", "exdate"])
	db.close()
	return optm

In [9]:
optm_df = l1.load_all_optm_data()

In [10]:
print(optm_df.shape)
print(optm_df['cp_flag'].value_counts().to_dict())

(3410580, 13)
{'P': 1706360, 'C': 1704220}


In [11]:
optm_df['date'] = pd.to_datetime(optm_df['date'])
optm_df['option_price'] = (optm_df['best_bid'] + optm_df['best_offer'])/2
optm_df['index_price'] = (optm_df['open'] + optm_df['close'])/2
optm_df['strike_price'] = optm_df['strike_price']/1000
optm_df['tb_m3'] = optm_df['tb_m3']/100

### Level 1 filters

#### Identical Filter:
The OptionMetrics data set contain duplicate observations,
defined as two or more quotes with identical option type, strike, expiration
date, and price. In each such case, we eliminate all but one of the quotes.

In [12]:
# Assuming 'df' is your DataFrame

def delete_identical_filter(df):
    columns_to_check = ['cp_flag', 'strike_price','date', 'exdate', 'best_offer']

    # Drop duplicates based on specified columns
    df = df.drop_duplicates(subset=columns_to_check, keep='first')

    return df

In [13]:
optm_l1_df = delete_identical_filter(optm_df)
print(optm_l1_df.shape)

(3410580, 15)


#### Identical Except Price Filter:
There are a few sets of quotes with identical
terms (type, strike, and maturity) but different prices. When this occurs, we
keep the quote whose T-bill-based implied volatility is closest to that of its
moneyness neighbors, and delete the others.

NEXT STEPS - determine moneyness neighbors >> only 18 duplicates so 9 deleted versus 11 in paper

In [14]:
def delete_identical_xprice_filter(df):
    columns_to_check = ['cp_flag', 'strike_price','date', 'exdate']

    # Drop duplicates based on specified columns
    df = df.drop_duplicates(subset=columns_to_check, keep='first')

    return df

In [15]:
columns_to_check = ['cp_flag', 'strike_price','date', 'exdate']

identical_df = optm_l1_df[optm_l1_df.duplicated(columns_to_check,keep=False)].sort_values(columns_to_check)

identical_df.shape

(20, 15)

In [16]:
identical_df = optm_l1_df[optm_l1_df.duplicated(columns_to_check,keep='first')].sort_values(columns_to_check)
identical_df.shape

(10, 15)

In [17]:
optm_l1_df = delete_identical_xprice_filter(optm_l1_df)
print(optm_l1_df.shape)

(3410570, 15)


In [18]:
def delete_zero_bid(df):
    df=df.loc[df['best_bid'] > 0]
    return df

In [19]:
optm_l1_df = delete_zero_bid(optm_l1_df)

In [20]:
optm_df.shape[0] - optm_l1_df.shape[0]

272088

### Level 2 filters
* “Days to Maturity <7 or >180” Filter
* “IV<5% or >100%” Filter
* “Moneyness <0.8 or >1.2” Filter
* “Implied Interest Rate <0” Filter

In [21]:
opmt_l2_df = optm_l1_df.copy()
opmt_l2_df.drop(columns=['secid'], inplace=True)

In [22]:
# Days to maturity <7 or >180
opmt_l2_df['time_to_maturity'] = (opmt_l2_df['exdate'] - opmt_l2_df['date']).dt.days

opmt_l2_df = opmt_l2_df.loc[(opmt_l2_df['time_to_maturity'] >= 7) & (opmt_l2_df['time_to_maturity'] <= 180)]

In [23]:
opmt_l2_df['time_to_maturity_yrs'] = opmt_l2_df['time_to_maturity']/365

In [24]:
test = opmt_l2_df.iloc[41964,:]

In [25]:
test.to_frame()

,66621
date,1996-11-12 00:00:00
open,731.87
close,729.56
cp_flag,P
exdate,1997-03-22 00:00:00
impl_volatility,0.155791
tb_m3,0.0504
volume,9.0
best_bid,18.25
best_offer,19.0


In [47]:
opmt_l2_df['tb_m3'].ffill(inplace=True)

In [20]:
bsm_pricer.calc_implied_volatility(18.625, 730.715, 720, 0.36, .0504, 'P')

0.16820384210428072

In [57]:
def calc_implied_volatility_wrapper(row):
    try:
        implied_volatility = bsm_pricer.calc_implied_volatility(row['option_price'], row['index_price'], row['strike_price'], row['time_to_maturity_yrs'], row['tb_m3'], row['cp_flag'])
        print(f"Implied Volatility {row.name}: {implied_volatility}")
        return implied_volatility
    except ValueError as e:
        # Handle the error as per your requirement, you can print the error or return a default value
        print(f"Error for row {row.name}: {e}")
        return 0

In [60]:
opmt_l2_df.reset_index(inplace=True)

,date,open,close,cp_flag,exdate,impl_volatility,tb_m3,volume,best_bid,best_offer,strike_price,contract_size,option_price,index_price,time_to_maturity,time_to_maturity_yrs
37,1996-01-04,621.32,617.70,C,1996-06-22,0.125173,0.0504,0.0,38.500,39.500,595.0,100.0,39.000,619.510,170,0.465753
37,1997-01-02,740.74,737.01,C,1997-01-18,0.832171,0.0505,0.0,188.500,189.500,550.0,100.0,189.000,738.875,16,0.043836
37,1998-01-02,970.43,975.04,C,1998-01-17,0.510554,0.0518,0.0,151.500,153.500,825.0,100.0,152.500,972.735,15,0.041096
37,1999-01-04,1229.23,1228.10,C,1999-06-19,0.337014,0.0438,0.0,226.500,228.500,1050.0,100.0,227.500,1228.665,166,0.454795
37,2000-01-03,1469.25,1455.22,C,2000-02-19,NaN,0.0527,0.0,310.625,312.625,1150.0,100.0,311.625,1462.235,47,0.128767
37,2001-01-02,1320.28,1283.27,P,2001-03-17,NaN,0.0569,0.0,344.625,346.625,1650.0,100.0,345.625,1301.775,74,0.202740
37,2002-01-02,1148.08,1154.67,C,2002-03-16,0.237305,0.0171,0.0,96.500,98.500,1075.0,100.0,97.500,1151.375,73,0.200000
37,2005-01-03,1211.92,1202.08,C,2005-01-22,0.507227,0.0229,0.0,194.300,196.300,1010.0,100.0,195.300,1207.000,19,0.052055
37,2006-01-03,1248.29,1268.80,P,2006-06-17,0.160385,0.0407,0.0,308.100,310.100,1600.0,100.0,309.100,1258.545,165,0.452055
37,2008-01-02,1467.97,1447.16,C,2008-03-31,0.236115,0.0318,0.0,69.300,71.300,1450.0,100.0,70.300,1457.565,89,0.243836


In [58]:
opmt_l2_df['impl_vol_bsm'] = opmt_l2_df.apply(calc_implied_volatility_wrapper, axis=1)

Implied Volatility 0: 0.05
Implied Volatility 1: 0.05
Implied Volatility 2: 0.05
Implied Volatility 3: 0.05
Implied Volatility 7: 0.05
Implied Volatility 8: 0.05
Implied Volatility 9: 0.05
Implied Volatility 10: 0.05
Implied Volatility 11: 0.05
Implied Volatility 12: 0.05
Implied Volatility 13: 0.05
Implied Volatility 14: 0.05
Implied Volatility 15: 0.05
Implied Volatility 17: 0.05
Implied Volatility 20: 0.05
Implied Volatility 21: 0.05
Implied Volatility 25: 0.05
Implied Volatility 26: 0.05
Implied Volatility 27: 0.05
Implied Volatility 28: 0.05
Implied Volatility 29: 0.05
Implied Volatility 30: 0.05
Implied Volatility 31: 0.05
Implied Volatility 34: 0.05
Implied Volatility 35: 0.05
Implied Volatility 37: 0.05147239096968619
Implied Volatility 38: 0.08340280548225575
Implied Volatility 39: 0.05
Implied Volatility 40: 0.05
Implied Volatility 41: 0.05
Implied Volatility 48: 0.13679256631026043
Implied Volatility 50: 0.05
Implied Volatility 51: 0.05
Implied Volatility 52: 0.0856389758311

KeyboardInterrupt: 

In [29]:
len(opmt_l2_df.loc[opmt_l2_df['impl_volatility'].isna()])

187052

In [32]:
opmt_l2_df.loc[(opmt_l2_df['impl_volatility']>=0.05) & (opmt_l2_df['impl_volatility']<=1.00)].shape[0]

1637279

In [28]:
opmt_l2_df.shape[0]

1840763

In [34]:
conn.get_table(library="wrdsapps", table="opcrsphist", obs=5)

,secid,sdate,edate,permno,score
0,5001.0,1996-01-02,1996-03-13,10074.0,1.0
1,5002.0,1996-01-01,1996-02-22,10154.0,1.0
2,5003.0,None,None,NaN,6.0
3,5004.0,1996-01-01,2000-01-27,80071.0,1.0
4,5005.0,1996-01-01,1997-08-12,85041.0,1.0


In [41]:
stdop = conn.get_table(library="optionm_all", table="stdopd1996")

In [42]:
stdop = stdop.loc[stdop['secid']==108105]

In [ ]:
df = pull_Option_info("hholt")

In [ ]:
opinfd.loc[opinfd['secid']==108105]

In [ ]:
df.head()

In [ ]:
dup_cols = ['cp_flag','strike_price']

duplicates = df[df.duplicated(subset=dup_cols, keep=False)]


In [ ]:
duplicates.head()